In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'

In [2]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [3]:
%%bash -s "$PLATFORM"
cd ../../../hardware/victims/firmware/bootloader-aes256/
make PLATFORM=$1 CRYPTO_TARGET=NONE

SS_VER set to SS_VER_1_1
rm -f -- bootloader-aes256-CWLITEXMEGA.hex
rm -f -- bootloader-aes256-CWLITEXMEGA.eep
rm -f -- bootloader-aes256-CWLITEXMEGA.cof
rm -f -- bootloader-aes256-CWLITEXMEGA.elf
rm -f -- bootloader-aes256-CWLITEXMEGA.map
rm -f -- bootloader-aes256-CWLITEXMEGA.sym
rm -f -- bootloader-aes256-CWLITEXMEGA.lss
rm -f -- objdir-CWLITEXMEGA/*.o
rm -f -- objdir-CWLITEXMEGA/*.lst
rm -f -- bootloader.s aes256.s crcccitt.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s
rm -f -- bootloader.d aes256.d crcccitt.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d
rm -f -- bootloader.i aes256.i crcccitt.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i
.
Welcome to another exciting ChipWhisperer target build!!
avr-gcc (GCC) 5.4.0
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE

In [4]:
fw_path = "../../../hardware/victims/firmware/bootloader-aes256/bootloader-aes256-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 4229 bytes


In [5]:
# Class Crc
#############################################################
# These CRC routines are copy-pasted from pycrc, which are:
# Copyright (c) 2006-2013 Thomas Pircher <tehpeh@gmx.net>
#
class Crc(object):
    """
    A base class for CRC routines.
    """

    def __init__(self, width, poly):
        """The Crc constructor.

        The parameters are as follows:
            width
            poly
            reflect_in
            xor_in
            reflect_out
            xor_out
        """
        self.Width = width
        self.Poly = poly


        self.MSB_Mask = 0x1 << (self.Width - 1)
        self.Mask = ((self.MSB_Mask - 1) << 1) | 1

        self.XorIn = 0x0000
        self.XorOut = 0x0000

        self.DirectInit = self.XorIn
        self.NonDirectInit = self.__get_nondirect_init(self.XorIn)
        if self.Width < 8:
            self.CrcShift = 8 - self.Width
        else:
            self.CrcShift = 0

    def __get_nondirect_init(self, init):
        """
        return the non-direct init if the direct algorithm has been selected.
        """
        crc = init
        for i in range(self.Width):
            bit = crc & 0x01
            if bit:
                crc ^= self.Poly
            crc >>= 1
            if bit:
                crc |= self.MSB_Mask
        return crc & self.Mask


    def bit_by_bit(self, in_data):
        """
        Classic simple and slow CRC implementation.  This function iterates bit
        by bit over the augmented input message and returns the calculated CRC
        value at the end.
        """
        # If the input data is a string, convert to bytes.
        if isinstance(in_data, str):
            in_data = [ord(c) for c in in_data]

        register = self.NonDirectInit
        for octet in in_data:
            for i in range(8):
                topbit = register & self.MSB_Mask
                register = ((register << 1) & self.Mask) | ((octet >> (7 - i)) & 0x01)
                if topbit:
                    register ^= self.Poly

        for i in range(self.Width):
            topbit = register & self.MSB_Mask
            register = ((register << 1) & self.Mask)
            if topbit:
                register ^= self.Poly

        return register ^ self.XorOut
    
bl_crc = Crc(width = 16, poly=0x1021)

In [6]:
hex((bl_crc.bit_by_bit([0x10]) >> 8) & 0xFF)

'0x12'

In [7]:
import time

In [8]:
ktp = cw.ktp.Basic()

In [14]:
okay = 0
while not okay:
    target.write('\0xxxxxxxxxxxxxxxxxx')
    time.sleep(0.005)
    response = target.read()
    if ord(response[0]) != 0xA4:
        okay = 1
target.read()

msg = [0x00]
key, text = ktp.new_pair()
msg.extend(text)
crc = bl_crc.bit_by_bit(text)
msg.append(crc >> 8)
msg.append(crc & 0xFF)

target.write(msg)
time.sleep(0.01)
response = target.read()
# print(response)
if(response):
    print("Response {:X}".format(ord(response[0])))



Response A4
